## Training einer konvolutionalen Layer anhand einfacher Produktionsdaten

### 1) Daten laden

In [1]:
import pandas as pd
from math import ceil

pd.set_option('display.max_columns', 12)

data_url=r'https://github.com/tplusone/hanser_ml_zeitreihen/blob/master/Daten/verlaufsdaten.csv?raw=true'
df = pd.read_csv(data_url)
df.head().round(1)

t0    t1    t2    t3    t4    t5    t6    t7  target
0  11.6  22.7  13.8  22.7  11.6  22.7  11.6  22.7       0
1   5.0   5.0  22.7   5.0   5.0   5.0   5.0   5.0       1
2   5.0   5.0   2.8  22.7   5.0   7.2   7.2   7.2       1
3   5.0   5.0   2.8   2.8  24.9   7.2   7.2   7.2       1
4   5.0   5.0   2.8   2.8   5.0  22.7   7.2   7.2       1

### 2) Vorbereitung der Daten inkl. Standardisierung und Restrukturierung
Achtung: Hier wird nicht spaltenweise standardisiert, sondern über die gesamten x-Daten!

In [2]:
y = df['target'].values
X = df.drop(['target'], axis=1).values

x_mean, x_std = X.mean(), X.std()
X = (X - x_mean) / x_std
X = X.reshape(-1, 8, 1)
X.shape, y.shape

((12, 8, 1), (12,))

### 3) Zusammenstellung des Modells

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Dense, Flatten,
                                     Conv1D, MaxPool1D)

model = Sequential(name='cnn')
model.add(Conv1D(filters=1, kernel_size=3, 
          input_shape=(8,1), activation='relu'))
model.add(MaxPool1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='sgd',
            metrics=['accuracy'])
model.summary()

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 6, 1)              4         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 3, 1)              0         
_________________________________________________________________
flatten (Flatten)            (None, 3)                 0         
_________________________________________________________________
dense (Dense)                (None, 1)                 4         
Total params: 8
Trainable params: 8
Non-trainable params: 0
_________________________________________________________________


### 4) Training des Modells
inkl. Evaluation

In [4]:
history = model.fit(X, y, epochs=1000, batch_size=1, 
                    verbose=False)
loss, acc = model.evaluate(X, y)
print('accuracy: {:.3}'.format(acc))
model.save('cnn_produktion_simple.h5')

12/12 [==============================] - 0s 6ms/sample - loss: 0.0224 - accuracy: 1.0000
accuracy: 1.0


### 5) Betrachtung der Gewichte

In [5]:
from tensorflow.keras.models import load_model

model = load_model('cnn_produktion_simple.h5')
w_conv = model.get_layer(index=0).get_weights()[0]
w_dense = model.get_layer(index=3).get_weights()[0]

print('weights, conv1:\n', w_conv)
print('weights, dense:\n',w_dense)

weights, conv1:
 [[[ 1.2854996 ]]

 [[-3.7454994 ]]

 [[ 0.07211278]]]
weights, dense:
 [[0.30993316]
 [3.0200465 ]
 [3.044641  ]]
